# CS 171 Final Project — Awards Model / Construction Notebook

**Goal:** Use 2025–26 per-game player stats to project:

- MVP
- Defensive Player of the Year (DPOY)
- Sixth Man of the Year (6MOY)
- All-NBA 1st / 2nd / 3rd teams
- All-Defensive 1st / 2nd teams
- All-Star selections (East / West)

**Data:**

- `players_25_26_awards_clean.csv` — rotation players only,
  with per-game stats, per-36 stats, positions, and teams.

**Tools (from CS171 lectures):**

- `numpy`, `pandas`
- `StandardScaler` (feature scaling)
- `KMeans` clustering (for “star tiers”)


In [40]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from IPython.display import display

pd.set_option("display.max_columns", 120)

DATA_DIR = "."
PLAYERS_PATH = f"{DATA_DIR}/players_25_26_awards_clean.csv"

players = pd.read_csv(PLAYERS_PATH)

print("Players shape:", players.shape)
players.head()


Players shape: (255, 38)


,Player,Age,Pos,Pos_simple,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional,Awards,PTS36,TRB36,AST36,STL36,BLK36,TOV36
0,Jalen Johnson,24.0,SF,F,ATL,20,20,34.9,8.4,15.5,0.539,1.7,4.0,0.413,6.7,11.5,0.583,0.592,4.9,6.2,0.789,1.3,8.8,10.0,7.3,1.6,0.4,3.3,2.1,23.2,johnsja05,NaN,23.931232,10.315186,7.530086,1.650430,0.412607,3.404011
1,Dyson Daniels,22.0,SG,G,ATL,22,22,33.8,4.6,9.5,0.488,0.2,1.4,0.161,4.4,8.1,0.545,0.500,0.8,1.4,0.548,2.2,4.3,6.5,6.0,2.3,0.4,2.3,2.5,10.3,daniedy01,NaN,10.970414,6.923077,6.390533,2.449704,0.426036,2.449704
2,Nickeil Alexander-Walker,27.0,SG,G,ATL,20,17,32.8,7.0,14.9,0.466,2.8,7.2,0.392,4.2,7.8,0.535,0.560,3.7,4.3,0.859,0.7,2.7,3.4,3.6,1.0,0.9,2.3,2.1,20.4,alexani01,NaN,22.390244,3.731707,3.951220,1.097561,0.987805,2.524390
3,Onyeka Okongwu,25.0,C,C,ATL,21,10,30.5,6.2,12.0,0.514,1.9,5.1,0.370,4.3,6.9,0.621,0.593,2.1,2.8,0.759,1.6,5.7,7.3,2.9,0.9,1.1,1.9,3.6,16.4,okongon01,NaN,19.357377,8.616393,3.422951,1.062295,1.298361,2.242623
4,Zaccharie Risacher,20.0,SF,F,ATL,19,19,24.7,4.5,10.0,0.453,1.5,4.8,0.308,3.1,5.2,0.586,0.526,0.9,1.4,0.630,0.6,2.0,2.6,1.5,1.1,0.7,0.8,2.1,11.4,risacza01,NaN,16.615385,3.789474,2.186235,1.603239,1.020243,1.165992


## Feature Groups

We define three groups of features:

- Offensive features: scoring, shooting, playmaking, minutes
- Defensive features: rebounding, steals, blocks, fouls
- Overall impact: combination of offensive and defensive scores

We automatically drop any feature that does not exist in the CSV.


In [41]:
# Offensive features
offense_features = [
    "PTS", "PTS36",
    "AST", "AST36",
    "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%",
    "MP"
]

# Defensive features
defense_features = [
    "TRB", "TRB36",
    "STL", "STL36",
    "BLK", "BLK36",
    "DRB", "ORB",
    "PF"
]

offense_features = [c for c in offense_features if c in players.columns]
defense_features = [c for c in defense_features if c in players.columns]

print("Offensive features:", offense_features)
print("Defensive features:", defense_features)


Offensive features: ['PTS', 'PTS36', 'AST', 'AST36', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'MP']
Defensive features: ['TRB', 'TRB36', 'STL', 'STL36', 'BLK', 'BLK36', 'DRB', 'ORB', 'PF']


## Standardize Features and Compute Z-scores

To combine stats measured on different scales (points, rebounds,
percentages, minutes), we standardize them using z-scores.

We use a helper function that:

- takes a list of feature columns,
- fills missing values with the column mean,
- standardizes them with `StandardScaler`,
- stores each standardized feature as a new column in the data frame.


In [42]:
def compute_z_scores(df, feature_list, prefix):
    """
    df: DataFrame with player rows
    feature_list: list of numeric feature column names
    prefix: prefix for new z-score columns

    Returns: (Z, updated_df)
    """
    X = df[feature_list].copy()
    X = X.fillna(X.mean())
    
    scaler = StandardScaler()
    Z = scaler.fit_transform(X)
    
    for i, col in enumerate(feature_list):
        z_col = f"{prefix}_{col}_z"
        df[z_col] = Z[:, i]
        
    return Z, df

Z_off, players = compute_z_scores(players, offense_features, prefix="off")
Z_def, players = compute_z_scores(players, defense_features, prefix="def")

players.head()


,Player,Age,Pos,Pos_simple,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional,Awards,PTS36,TRB36,AST36,STL36,BLK36,TOV36,off_PTS_z,off_PTS36_z,off_AST_z,off_AST36_z,off_FG%_z,off_3P_z,off_3PA_z,off_3P%_z,off_FT_z,off_FTA_z,off_FT%_z,off_MP_z,def_TRB_z,def_TRB36_z,def_STL_z,def_STL36_z,def_BLK_z,def_BLK36_z,def_DRB_z,def_ORB_z,def_PF_z
0,Jalen Johnson,24.0,SF,F,ATL,20,20,34.9,8.4,15.5,0.539,1.7,4.0,0.413,6.7,11.5,0.583,0.592,4.9,6.2,0.789,1.3,8.8,10.0,7.3,1.6,0.4,3.3,2.1,23.2,johnsja05,NaN,23.931232,10.315186,7.530086,1.650430,0.412607,3.404011,1.533875,1.208322,2.232311,1.678406,0.817591,0.232321,-0.031475,0.687340,1.439644,1.527810,0.133180,1.427425,2.194048,1.294727,1.702226,0.752241,-0.246363,-0.473610,2.971882,0.081407,-0.226474
1,Dyson Daniels,22.0,SG,G,ATL,22,22,33.8,4.6,9.5,0.488,0.2,1.4,0.161,4.4,8.1,0.545,0.500,0.8,1.4,0.548,2.2,4.3,6.5,6.0,2.3,0.4,2.3,2.5,10.3,daniedy01,NaN,10.970414,6.923077,6.390533,2.449704,0.426036,2.449704,-0.435633,-1.161902,1.563646,1.147333,0.167030,-1.377514,-1.153395,-1.969017,-0.791196,-0.654266,-2.059309,1.247295,0.730311,0.133427,3.444470,2.325908,-0.246363,-0.452953,0.435071,1.085858,0.397859
2,Nickeil Alexander-Walker,27.0,SG,G,ATL,20,17,32.8,7.0,14.9,0.466,2.8,7.2,0.392,4.2,7.8,0.535,0.560,3.7,4.3,0.859,0.7,2.7,3.4,3.6,1.0,0.9,2.3,2.1,20.4,alexani01,NaN,22.390244,3.731707,3.951220,1.097561,0.987805,2.524390,1.106385,0.926512,0.329189,0.010525,-0.113604,1.412868,1.349350,0.465977,0.786715,0.664071,0.770002,1.083541,-0.566143,-0.959148,0.208874,-0.336285,0.848106,0.411262,-0.466906,-0.588228,-0.226474
3,Onyeka Okongwu,25.0,C,C,ATL,21,10,30.5,6.2,12.0,0.514,1.9,5.1,0.370,4.3,6.9,0.621,0.593,2.1,2.8,0.759,1.6,5.7,7.3,2.9,0.9,1.1,1.9,3.6,16.4,okongon01,NaN,19.357377,8.616393,3.422951,1.062295,1.298361,2.242623,0.495685,0.371873,-0.030861,-0.235667,0.498688,0.446966,0.443184,0.234073,-0.083857,-0.017827,-0.139744,0.706906,1.064879,0.713139,-0.040018,-0.405719,1.285893,0.889014,1.224301,0.416224,2.114774
4,Zaccharie Risacher,20.0,SF,F,ATL,19,19,24.7,4.5,10.0,0.453,1.5,4.8,0.308,3.1,5.2,0.586,0.526,0.9,1.4,0.630,0.6,2.0,2.6,1.5,1.1,0.7,0.8,2.1,11.4,risacza01,NaN,16.615385,3.789474,2.186235,1.603239,1.020243,1.165992,-0.267690,-0.129572,-0.750962,-0.812022,-0.279434,0.017677,0.313731,-0.419475,-0.736785,-0.654266,-1.313317,-0.242870,-0.900711,-0.939372,0.457766,0.659329,0.410318,0.461164,-0.861521,-0.699834,-0.226474


## Offensive, Defensive, and Overall Scores plus KMeans Tiers

We now build:

- `off_score` = mean of offensive z-features
- `def_score` = mean of defensive z-features
- `overall_score` = 0.6 * off_score + 0.4 * def_score

We then train two KMeans models (k = 3):

- Offensive KMeans on offensive z-features
- Defensive KMeans on defensive z-features

We map the raw cluster labels so that:

- tier 0 = strongest cluster (highest mean score)
- tier 1 = middle cluster
- tier 2 = lower cluster


In [43]:
off_z_cols = [c for c in players.columns if c.startswith("off_") and c.endswith("_z")]
def_z_cols = [c for c in players.columns if c.startswith("def_") and c.endswith("_z")]

players["off_score"] = players[off_z_cols].mean(axis=1)
players["def_score"] = players[def_z_cols].mean(axis=1)
players["overall_score"] = 0.6 * players["off_score"] + 0.4 * players["def_score"]

# Train KMeans models
k_off = 3
k_def = 3

kmeans_off = KMeans(n_clusters=k_off, random_state=0, n_init=10)
off_clusters = kmeans_off.fit_predict(Z_off)

kmeans_def = KMeans(n_clusters=k_def, random_state=0, n_init=10)
def_clusters = kmeans_def.fit_predict(Z_def)

players["off_cluster_raw"] = off_clusters
players["def_cluster_raw"] = def_clusters

# Map raw clusters to tiers (0 = best)
off_cluster_means = players.groupby("off_cluster_raw")["off_score"].mean()
def_cluster_means = players.groupby("def_cluster_raw")["def_score"].mean()

off_cluster_order = off_cluster_means.sort_values(ascending=False).index.tolist()
def_cluster_order = def_cluster_means.sort_values(ascending=False).index.tolist()

off_cluster_to_tier = {cluster: tier for tier, cluster in enumerate(off_cluster_order)}
def_cluster_to_tier = {cluster: tier for tier, cluster in enumerate(def_cluster_order)}

players["off_tier"] = players["off_cluster_raw"].map(off_cluster_to_tier)
players["def_tier"] = players["def_cluster_raw"].map(def_cluster_to_tier)

players[["Player","Tm","Pos_simple","off_score","off_tier","def_score","def_tier","overall_score"]].head(10)


C:\Users\hassa\miniconda3\envs\cs171\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\hassa\miniconda3\envs\cs171\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,Player,Tm,Pos_simple,off_score,off_tier,def_score,def_tier,overall_score
0,Jalen Johnson,ATL,F,1.073896,0,0.894454,0,1.002119
1,Dyson Daniels,ATL,G,-0.456411,2,0.872621,1,0.075202
2,Nickeil Alexander-Walker,ATL,G,0.732628,0,-0.186105,2,0.365135
3,Onyeka Okongwu,ATL,C,0.224118,1,0.806943,0,0.457248
4,Zaccharie Risacher,ATL,F,-0.439582,1,-0.182148,1,-0.336609
5,Vit Krejci,ATL,G,-0.165815,1,-0.700082,2,-0.379522
6,Luke Kennard,ATL,G,-0.238654,1,-0.859300,2,-0.486912
7,Mouhamed Gueye,ATL,F,-0.969085,2,-0.058941,2,-0.605028
8,Jaylen Brown,BOS,F,1.066620,0,0.240960,1,0.736356
9,Derrick White,BOS,G,0.501502,1,0.392523,1,0.457910


## Starter and Bench Flags

Many awards depend on a player's role.

We define:

- `start_rate` = fraction of games started
- `is_starter` = start_rate at least 0.5
- `is_bench`   = start_rate at most 0.4


In [44]:
players["start_rate"] = players["GS"] / players["G"]

players["is_starter"] = players["start_rate"] >= 0.5
players["is_bench"]   = players["start_rate"] <= 0.4

players[["Player","Tm","G","GS","start_rate","is_starter","is_bench"]].head()


,Player,Tm,G,GS,start_rate,is_starter,is_bench
0,Jalen Johnson,ATL,20,20,1.00000,True,False
1,Dyson Daniels,ATL,22,22,1.00000,True,False
2,Nickeil Alexander-Walker,ATL,20,17,0.85000,True,False
3,Onyeka Okongwu,ATL,21,10,0.47619,False,False
4,Zaccharie Risacher,ATL,19,19,1.00000,True,False


## MVP Selection

We define an MVP score as a weighted combination of:

- offensive impact (`off_score`)
- defensive impact (`def_score`)
- minutes per game (standardized as `MP_z`)

Formula (described in words):

- mvp_score = 0.7 * off_score + 0.3 * def_score + 0.2 * MP_z

MVP candidates:

- starters (`is_starter == True`)
- in one of the top two offensive tiers (`off_tier <= 1`)
- at least 20 minutes per game (`MP >= 20`)

We rank candidates by `mvp_score` and take the top one as MVP.
If the filter is ever empty, we fall back to all players to avoid errors.


In [46]:
# Standardize MP
mp_scaler = StandardScaler()
mp_z = mp_scaler.fit_transform(players[["MP"]].fillna(players["MP"].mean()))
players["MP_z"] = mp_z[:, 0]

# MVP score
players["mvp_score"] = (
    0.7 * players["off_score"] +
    0.3 * players["def_score"] +
    0.2 * players["MP_z"]
)

mvp_candidates = players[
    (players["is_starter"]) &
    (players["off_tier"] <= 1) &
    (players["MP"] >= 20)
].copy()

if mvp_candidates.empty:
    print("Warning: MVP candidate filter empty, falling back to all players.")
    mvp_candidates = players.copy()

mvp_candidates = mvp_candidates.sort_values("mvp_score", ascending=False)

mvp_winner = mvp_candidates.iloc[0]

print("=== MVP Projection ===")
print(
    mvp_winner[
        ["Player","Tm","Pos_simple","G","MP","PTS",
         "off_score","def_score","overall_score","mvp_score"]
    ]
)


=== MVP Projection ===
Player           Luka Dončić
Tm                       LAL
Pos_simple                 G
G                         16
MP                      36.9
PTS                     35.3
off_score            2.27494
def_score           0.820716
overall_score        1.69325
mvp_score            2.18966
Name: 97, dtype: object


## Defensive Player of the Year (DPOY)

DPOY focuses on defensive impact and minutes.

We define:

- dpoy_score = def_score + 0.2 * MP_z

DPOY candidates:

- at least 20 minutes per game (`MP >= 20`)

We rank by `dpoy_score` and take the top player as DPOY.


In [47]:
players["dpoy_score"] = players["def_score"] + 0.2 * players["MP_z"]

dpoy_candidates = players[players["MP"] >= 20].copy()

if dpoy_candidates.empty:
    print("Warning: DPOY candidate filter empty, falling back to all players.")
    dpoy_candidates = players.copy()

dpoy_candidates = dpoy_candidates.sort_values("dpoy_score", ascending=False)

dpoy_winner = dpoy_candidates.iloc[0]

print("=== DPOY Projection ===")
print(
    dpoy_winner[
        ["Player","Tm","Pos_simple","G","MP",
         "TRB","STL","BLK","def_score","dpoy_score"]
    ]
)


=== DPOY Projection ===
Player        Victor Wembanyama
Tm                          SAS
Pos_simple                    C
G                            12
MP                         34.7
TRB                        12.9
STL                         1.1
BLK                         3.6
def_score              2.738238
dpoy_score             3.017172
Name: 216, dtype: object


## Sixth Man of the Year (6MOY)

Sixth Man is a bench award, so candidates:

- must be bench players (`is_bench == True`)
- must play at least 18 minutes per game (`MP >= 18`)

We define:

- sixth_score = off_score + 0.1 * def_score

We rank bench players by `sixth_score` and take the top one as
Sixth Man of the Year.


In [48]:
players["sixth_score"] = players["off_score"] + 0.1 * players["def_score"]

sixth_candidates = players[
    (players["is_bench"]) &
    (players["MP"] >= 18)
].copy()

if sixth_candidates.empty:
    print("Warning: Sixth Man candidate filter empty, falling back to all bench players.")
    sixth_candidates = players[players["is_bench"]].copy()

sixth_candidates = sixth_candidates.sort_values("sixth_score", ascending=False)

sixth_winner = sixth_candidates.iloc[0]

print("=== Sixth Man of the Year Projection ===")
print(
    sixth_winner[
        ["Player","Tm","Pos_simple","G","MP","PTS",
         "off_score","sixth_score"]
    ]
)


=== Sixth Man of the Year Projection ===
Player         Jerami Grant
Tm                      POR
Pos_simple                F
G                        20
MP                     29.2
PTS                    19.1
off_score          0.648043
sixth_score        0.639127
Name: 200, dtype: object


## All-NBA Teams

We build All-NBA First, Second, and Third teams.

Each team has:

- 2 guards (G)
- 2 forwards (F)
- 1 center (C)

Candidate pool:

- at least 20 minutes per game (`MP >= 20`)

Teams are chosen by sorting players by `overall_score` and filling
position slots in order.


In [49]:
def build_team(sorted_df, taken_players, g_needed=2, f_needed=2, c_needed=1):
    team = []

    def pick(pos, n):
        nonlocal team, taken_players
        candidates = sorted_df[
            (sorted_df["Pos_simple"] == pos) &
            (~sorted_df["Player"].isin(taken_players))
        ]
        chosen = candidates.head(n)
        team.append(chosen)
        taken_players.update(chosen["Player"].tolist())

    pick("G", g_needed)
    pick("F", f_needed)
    pick("C", c_needed)

    if team:
        return pd.concat(team)
    else:
        return pd.DataFrame(columns=sorted_df.columns)


nba_pool = players[players["MP"] >= 20].copy()

if nba_pool.empty:
    print("Warning: All-NBA pool empty, falling back to all players.")
    nba_pool = players.copy()

nba_pool = nba_pool.sort_values("overall_score", ascending=False)

taken = set()

all_nba_first  = build_team(nba_pool, taken)
all_nba_second = build_team(nba_pool, taken)
all_nba_third  = build_team(nba_pool, taken)

print("=== All-NBA First Team ===")
display(all_nba_first[["Player","Tm","Pos_simple","overall_score"]])

print("=== All-NBA Second Team ===")
display(all_nba_second[["Player","Tm","Pos_simple","overall_score"]])

print("=== All-NBA Third Team ===")
display(all_nba_third[["Player","Tm","Pos_simple","overall_score"]])


=== All-NBA First Team ===


,Player,Tm,Pos_simple,overall_score
97,Luka Dončić,LAL,G,1.693250
178,Tyrese Maxey,PHI,G,1.200462
126,Giannis Antetokounmpo,MIL,F,1.306054
0,Jalen Johnson,ATL,F,1.002119
62,Nikola Jokić,DEN,C,1.800838


=== All-NBA Second Team ===


,Player,Tm,Pos_simple,overall_score
160,Shai Gilgeous-Alexander,OKC,G,1.129049
89,James Harden,LAC,G,1.059756
197,Deni Avdija,POR,F,0.907835
227,Scottie Barnes,TOR,F,0.879021
216,Victor Wembanyama,SAS,C,1.663845


=== All-NBA Third Team ===


,Player,Tm,Pos_simple,overall_score
43,Donovan Mitchell,CLE,G,0.974211
98,Austin Reaves,LAL,G,0.946952
91,Kawhi Leonard,LAC,F,0.807018
8,Jaylen Brown,BOS,F,0.736356
71,Alperen Şengün,HOU,C,0.988450


## All-Defensive Teams

We build All-Defensive First and Second teams with the same position
structure (2 guards, 2 forwards, 1 center).

Candidate pool:

- at least 18 minutes per game (`MP >= 18`)

Teams are chosen by sorting players by `def_score`.


In [50]:
def_pool = players[players["MP"] >= 18].copy()

if def_pool.empty:
    print("Warning: All-Defensive pool empty, falling back to all players.")
    def_pool = players.copy()

def_pool = def_pool.sort_values("def_score", ascending=False)

taken_def = set()

all_def_first  = build_team(def_pool, taken_def)
all_def_second = build_team(def_pool, taken_def)

print("=== All-Defensive First Team ===")
display(all_def_first[["Player","Tm","Pos_simple","def_score"]])

print("=== All-Defensive Second Team ===")
display(all_def_second[["Player","Tm","Pos_simple","def_score"]])


=== All-Defensive First Team ===


,Player,Tm,Pos_simple,def_score
1,Dyson Daniels,ATL,G,0.872621
97,Luka Dončić,LAL,G,0.820716
126,Giannis Antetokounmpo,MIL,F,1.325579
227,Scottie Barnes,TOR,F,1.254664
216,Victor Wembanyama,SAS,C,2.738238


=== All-Defensive Second Team ===


,Player,Tm,Pos_simple,def_score
175,Jalen Suggs,ORL,G,0.656486
219,Stephon Castle,SAS,G,0.649563
44,Evan Mobley,CLE,F,1.076627
161,Chet Holmgren,OKC,F,0.908126
62,Nikola Jokić,DEN,C,1.706229


## All-Star Selections (East and West)

We select All-Stars by conference:

- Map teams to East or West
- Candidate pool: at least 20 minutes per game and known conference
- Select up to 12 All-Stars per conference by `overall_score`


In [52]:
east_teams = ['ATL','BOS','BKN','CHA','CHI','CLE','DET','IND',
              'MIA','MIL','NYK','ORL','PHI','TOR','WAS']
west_teams = ['DAL','DEN','GSW','HOU','LAC','LAL','MEM','MIN',
              'NOP','OKC','PHX','POR','SAC','SAS','UTA']

players["Conference"] = np.where(
    players["Tm"].isin(east_teams), "East",
    np.where(players["Tm"].isin(west_teams), "West", "Unknown")
)

allstar_pool = players[
    (players["MP"] >= 20) &
    (players["Conference"] != "Unknown")
].copy()

if allstar_pool.empty:
    print("Warning: All-Star pool empty, falling back to all players with known conference.")
    allstar_pool = players[players["Conference"] != "Unknown"].copy()

east_pool = allstar_pool[allstar_pool["Conference"]=="East"].sort_values("overall_score", ascending=False)
west_pool = allstar_pool[allstar_pool["Conference"]=="West"].sort_values("overall_score", ascending=False)

EAST_N = min(12, len(east_pool))
WEST_N = min(12, len(west_pool))

east_allstars = east_pool.head(EAST_N)
west_allstars = west_pool.head(WEST_N)

print("=== East All-Stars ===")
display(east_allstars[["Player","Tm","Pos_simple","overall_score"]])

print("=== West All-Stars ===")
display(west_allstars[["Player","Tm","Pos_simple","overall_score"]])


=== East All-Stars ===


,Player,Tm,Pos_simple,overall_score
126,Giannis Antetokounmpo,MIL,F,1.306054
178,Tyrese Maxey,PHI,G,1.200462
0,Jalen Johnson,ATL,F,1.002119
43,Donovan Mitchell,CLE,G,0.974211
153,Karl-Anthony Towns,NYK,C,0.971726
26,Josh Giddey,CHI,G,0.880130
227,Scottie Barnes,TOR,F,0.879021
8,Jaylen Brown,BOS,F,0.736356
37,LaMelo Ball,CHA,G,0.706596
44,Evan Mobley,CLE,F,0.661264


=== West All-Stars ===


,Player,Tm,Pos_simple,overall_score
62,Nikola Jokić,DEN,C,1.800838
97,Luka Dončić,LAL,G,1.693250
216,Victor Wembanyama,SAS,C,1.663845
160,Shai Gilgeous-Alexander,OKC,G,1.129049
89,James Harden,LAC,G,1.059756
71,Alperen Şengün,HOU,C,0.988450
98,Austin Reaves,LAL,G,0.946952
197,Deni Avdija,POR,F,0.907835
133,Anthony Edwards,MIN,G,0.808861
91,Kawhi Leonard,LAC,F,0.807018


## Awards Summary

We summarize all final award picks made by the model:

- MVP
- DPOY
- Sixth Man
- All-NBA 1st / 2nd / 3rd
- All-Defensive 1st / 2nd
- East and West All-Star teams


In [53]:
print("===== AWARDS SUMMARY =====\n")

print("MVP:", mvp_winner["Player"], "-", mvp_winner["Tm"])
print("DPOY:", dpoy_winner["Player"], "-", dpoy_winner["Tm"])
print("Sixth Man:", sixth_winner["Player"], "-", sixth_winner["Tm"])
print()

print("All-NBA First Team:")
for _, row in all_nba_first.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")

print("\nAll-NBA Second Team:")
for _, row in all_nba_second.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")

print("\nAll-NBA Third Team:")
for _, row in all_nba_third.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")

print("\nAll-Defensive First Team:")
for _, row in all_def_first.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")

print("\nAll-Defensive Second Team:")
for _, row in all_def_second.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")

print("\nEast All-Stars:")
for _, row in east_allstars.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")

print("\nWest All-Stars:")
for _, row in west_allstars.iterrows():
    print(f"  {row['Pos_simple']} - {row['Player']} ({row['Tm']})")


===== AWARDS SUMMARY =====

MVP: Luka Dončić - LAL
DPOY: Victor Wembanyama - SAS
Sixth Man: Jerami Grant - POR

All-NBA First Team:
  G - Luka Dončić (LAL)
  G - Tyrese Maxey (PHI)
  F - Giannis Antetokounmpo (MIL)
  F - Jalen Johnson (ATL)
  C - Nikola Jokić (DEN)

All-NBA Second Team:
  G - Shai Gilgeous-Alexander (OKC)
  G - James Harden (LAC)
  F - Deni Avdija (POR)
  F - Scottie Barnes (TOR)
  C - Victor Wembanyama (SAS)

All-NBA Third Team:
  G - Donovan Mitchell (CLE)
  G - Austin Reaves (LAL)
  F - Kawhi Leonard (LAC)
  F - Jaylen Brown (BOS)
  C - Alperen Şengün (HOU)

All-Defensive First Team:
  G - Dyson Daniels (ATL)
  G - Luka Dončić (LAL)
  F - Giannis Antetokounmpo (MIL)
  F - Scottie Barnes (TOR)
  C - Victor Wembanyama (SAS)

All-Defensive Second Team:
  G - Jalen Suggs (ORL)
  G - Stephon Castle (SAS)
  F - Evan Mobley (CLE)
  F - Chet Holmgren (OKC)
  C - Nikola Jokić (DEN)

East All-Stars:
  F - Giannis Antetokounmpo (MIL)
  G - Tyrese Maxey (PHI)
  F - Jalen Johnso